In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, time
from sklearn.cluster import DBSCAN,KMeans
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from functools import reduce

In [ ]:
# reading the data file
df = pd.read_csv('testing_data_chennai.csv')
display(df)
X = df.to_numpy()

,lng,lat
0,80.222310,13.074261
1,80.218900,13.077467
2,80.220347,13.075223
3,80.223502,13.073548
4,80.222569,13.076430
...,...,...
1895,80.194569,13.113041
1896,80.219928,13.130711
1897,80.248042,13.114009
1898,80.257501,13.131415


In [ ]:
kms_per_radian = 6371.0088

epsilon = 0.28 / kms_per_radian # 40 mitre cluster
db = DBSCAN(eps=epsilon, min_samples=10, algorithm='ball_tree', metric='euclidean').fit(np.radians(X))

cluster_labels = db.labels_
final = np.concatenate((X,cluster_labels.reshape(-1,1)),axis=1)
print(final)

num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
num_noise_ = list(cluster_labels).count(-1)
clusters = pd.Series([X[cluster_labels == n] for n in range(num_clusters)])
print('Number of clusters: {}'.format(num_clusters))
print('Number of noise: {}'.format(num_noise_))

[[80.22231013 13.07426133  0.        ]
 [80.21890039 13.07746726  1.        ]
 [80.22034684 13.07522299  0.        ]
 ...
 [80.2480417  13.11400878 -1.        ]
 [80.25750136 13.13141513 -1.        ]
 [80.21881666 13.12062508 -1.        ]]
Number of clusters: 12
Number of noise: 573


In [ ]:
X_exc_noise=[]
for i in range(len(cluster_labels)):
      if cluster_labels[i] != -1:
          X_exc_noise.append(X[i])
X_exc_noise=np.array(X_exc_noise)
print(len(X_exc_noise))
kmeans = KMeans(n_clusters = num_clusters, init ='k-means++',random_state=3)
kmeans.fit(X_exc_noise) # Compute k-means clustering.
cluster_labels=kmeans.labels_
final = np.concatenate((X_exc_noise,cluster_labels.reshape(-1,1)),axis=1)
print(final)
np.savetxt("corresponding_clusters_chennai.csv",final,fmt='%3.5f',delimiter=',')
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
num_noise_ = list(cluster_labels).count(-1)
clusters = pd.Series([X_exc_noise[cluster_labels == n] for n in range(num_clusters)])
print('Number of clusters: {}'.format(num_clusters))
print('Number of noise: {}'.format(num_noise_))

1327
[[80.22231013 13.07426133  9.        ]
 [80.21890039 13.07746726  9.        ]
 [80.22034684 13.07522299  9.        ]
 ...
 [80.2817046  13.1281877   6.        ]
 [80.2843912  13.11690463  6.        ]
 [80.23605072 13.11880548  1.        ]]
Number of clusters: 12
Number of noise: 0


In [ ]:
centroidspts=kmeans.cluster_centers_
np.savetxt("centroids_chennai.csv",centroidspts,fmt='%2.5f',delimiter=',')

In [ ]:
# Graham Scan Algorithm
from matplotlib import pyplot as plt
from random import randint
from math import atan2

def scatter_plot(coords,convex_hull=None):
    xs,ys=zip(*coords)
    plt.scatter(xs,ys) 

    if convex_hull!=None:

        for i in range(1,len(convex_hull)+1):
            if i==len(convex_hull): i=0 # wrap
            c0=convex_hull[i-1]
            c1=convex_hull[i]
            plt.plot((c0[0],c1[0]),(c0[1],c1[1]),'r')
    plt.show()


def polar_angle(p0,p1=None):
    if p1==None: p1=anchor
    y_span=p0[1]-p1[1]
    x_span=p0[0]-p1[0]
    return atan2(y_span,x_span)

def distance(p0,p1=None):
    if p1==None: p1=anchor
    y_span=p0[1]-p1[1]
    x_span=p0[0]-p1[0]
    return y_span**2 + x_span**2

def det(p1,p2,p3):
    return   (p2[0]-p1[0])*(p3[1]-p1[1]) \
            -(p2[1]-p1[1])*(p3[0]-p1[0])


def quicksort(a):
    if len(a)<=1: return a
    smaller,equal,larger=[],[],[]
    piv_ang=polar_angle(a[randint(0,len(a)-1)]) # select random pivot
    for pt in a:
        pt_ang=polar_angle(pt) # calculate current point angle
        if   pt_ang<piv_ang:  smaller.append(pt)
        elif pt_ang==piv_ang: equal.append(pt)
        else: 				  larger.append(pt)
    return   quicksort(smaller) \
            +sorted(equal,key=distance) \
            +quicksort(larger)


def graham_scan(points,show_progress=False):
    points = points.tolist()
    global anchor
    min_idx=None
    for i,(x,y) in enumerate(points):
        if min_idx==None or y<points[min_idx][1]:
            min_idx=i
        if y==points[min_idx][1] and x<points[min_idx][0]:
            min_idx=i

    anchor=points[min_idx]

    sorted_pts=quicksort(points)
    del sorted_pts[sorted_pts.index(anchor)]

    hull=[anchor,sorted_pts[0]]
    for s in sorted_pts[1:]:
        while det(hull[-2],hull[-1],s)<=0:
            del hull[-1] 
        hull.append(s)
        if show_progress: scatter_plot(points,hull)
    return hull


boundary_points = []
for i in range(num_clusters):
    hull = graham_scan(clusters[i])
    boundary_points.append(hull)

boundary_points = pd.Series(boundary_points)
boundary_points.to_csv("boundary_chennai.csv")
